In [9]:
from graphdatascience import GraphDataScience

import json

# neo4j desktop (5.3.0, m1)

In [10]:
host = "bolt://localhost:7687"
user = "neo4j"
password= "j4oenj4oen"

gds = GraphDataScience(host, auth=(user, password))
print(gds.version())

2.3.2


# Create Noun-Article Graph

In [11]:
with open("data/data.json", "r") as file:
    data = json.load(file)
for key in data:
    for element0 in data[key]:
        for element1 in data[key][element0]:
            node = element0 + "-" + element1
            if "tfidf" not in data[key][element0][element1]:
                query = """
                MERGE (n:Noun {name:$key})
                MERGE (a:Article {name:$node, lines:$lines})
                MERGE (n)-[l:APPEAR_IN]-(a)
                """
                params = {'key': key, 'node': node, 'lines': data[key][element0][element1]["lines"]}
            else:
                query = """
                MERGE (n:Noun {name:$key})
                MERGE (a:Article {name:$node, lines:$lines, tfidf:$tfidf})
                MERGE (n)-[l:APPEAR_IN]-(a)
                """
                params = {'key': key, 'node': node, 'lines': data[key][element0][element1]["lines"], 'tfidf': data[key][element0][element1]["tfidf"]}
            gds.run_cypher(query, params)

# (postprocessing) free up memory

In [ ]:
query = """
MATCH (n) DETACH DELETE n
"""
gds.run_cypher(query)
gds.close()